In [2]:
from datasets import load_dataset
raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

Found cached dataset kde4 (C:/Users/Raj/.cache/huggingface/datasets/kde4/en-fr-lang1=en,lang2=fr/0.0.0/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [4]:
# this one train dataset needs to be split into train and validation
split_datasets = raw_datasets["train"].train_test_split(test_size=0.1, seed = 20)
split_datasets

Loading cached split indices for dataset at C:\Users\Raj\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-a067345757f9ef23.arrow and C:\Users\Raj\.cache\huggingface\datasets\kde4\en-fr-lang1=en,lang2=fr\0.0.0\243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac\cache-22b7e661839e4670.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [5]:
# we can rename the "test" key to "validation" like this
split_datasets["validation"] = split_datasets.pop("test")
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [6]:
# Let's take a look at one element of the dataset
example = split_datasets["train"][0]
print(example)

{'id': '92924', 'translation': {'en': "Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.", 'fr': "Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape."}}


In [8]:
from IPython.display import display, Markdown, JSON
display(Markdown("> **English**\n> {}".format(example["translation"]["en"])))
display(Markdown("> **French**\n> {}".format(example["translation"]["fr"])))

> **English**
> Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.

> **French**
> Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape.

In [11]:
display(JSON(example))

<IPython.core.display.JSON object>

In [12]:
from pprint import pprint
pprint(example)

{'id': '92924',
 'translation': {'en': 'Calibration is about to check the value range your '
                       'device delivers. Please move axis %1 %2 on your device '
                       'to the maximum position. Press any button on the '
                       "device or click on the'Next 'button to continue with "
                       'the next step.',
                 'fr': 'Le calibrage va vérifier la plage de valeurs que votre '
                       "matériel produit. Veuillez déplacer l'axe %1 %2 de "
                       'votre périphérique à la position maximale. Appuyez sur '
                       "n'importe quel bouton du périphérique ou sur le bouton "
                       '« & #160; Suivant & #160; » pour la prochaine étape.'}}


In [14]:
import json
print(json.dumps(example, indent=2, sort_keys=True))

{
  "id": "92924",
  "translation": {
    "en": "Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.",
    "fr": "Le calibrage va v\u00e9rifier la plage de valeurs que votre mat\u00e9riel produit. Veuillez d\u00e9placer l'axe %1 %2 de votre p\u00e9riph\u00e9rique \u00e0 la position maximale. Appuyez sur n'importe quel bouton du p\u00e9riph\u00e9rique ou sur le bouton \u00ab & #160; Suivant & #160; \u00bb pour la prochaine \u00e9tape."
  }
}


In [18]:
JSON(example)

<IPython.core.display.JSON object>

In [16]:
example

{'id': '92924',
 'translation': {'en': "Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.",
  'fr': "Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape."}}

In [19]:
# Testing the behavior of the translation - shows lazy translation - leaves computer science related words in english while translating the rest to french

from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint, tokenizer=model_checkpoint)
translator("Default to expanded threads")

c:\Users\Raj\repos\hf-nlp\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Par défaut pour les threads élargis'}]

In [20]:
# another example of this behavior can be seen with the word plugin
split_datasets["train"][172]["translation"]


{'en': 'Unable to import %1 using the OFX importer plugin. This file is not the correct format.',
 'fr': "Impossible d'importer %1 en utilisant le module d'extension d'importation OFX. Ce fichier n'a pas un format correct."}

In [21]:
translator(split_datasets["train"][172]["translation"]["en"])

[{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}]

In [23]:
# similar behavior with email translation
translator("Write me an email soon")

[{'translation_text': 'Ecrivez-moi un e-mail bientôt'}]

In [24]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

c:\Users\Raj\repos\hf-nlp\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [26]:
# Data prep: remember that the tokenizer processes the target in the output language, here French, so we need to set the source and target languages accordingly
# to see how this work, let's process one sample of each language in the training set
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target = fr_sentence)
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [27]:
en_sentence, fr_sentence

('Default to expanded threads',
 'Par défaut, développer les fils de discussion')

In [30]:
# if we forget to indicate that we are tokenizing labels, they will be tokenized by the input tokenizer, which in the case of a Marian model is not going to go well at all

wrong_targets = tokenizer(fr_sentence)
print(wrong_targets)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

{'input_ids': [577, 1205, 483, 13077, 2, 1205, 517, 13926, 1588, 16, 3842, 9, 5, 1710, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [33]:
# Define preprocessing function on the dataset
max_length = 128

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, 
                             text_target=targets, 
                             max_length=max_length, 
                             truncation=True)

    return model_inputs

In [34]:
# we can now apply that preprocessing function to our datasets in one go on all the splits
tokenized_datasets = split_datasets.map(preprocess_function, 
                                        batched=True,
                                        remove_columns=split_datasets["train"].column_names,
                                        )

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

In [35]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21018
    })
})

In [36]:
# We will start with the AutoModel to fine tune; There will be no warning about missing weights or newly initialized weights, because the model was trained on a translation task
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Next we will need a data collator to deal with the padding for dynamic batching. We can't just use the DataCollatorWithPadding because that only pads the inputs (inputIDs, attention mask, and token type IDs). Our labels should also be padded to the maximum length encountered in the labels. The padding value used to pad the labels should be -100 and not the padding token of the tokenizer, to make sure those padded values are ignored in the loss computation.

In [37]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [42]:
# to test this on a few samples, we just call it on a list of examples from our tokkenized training set:

batch = data_collator([tokenized_datasets["train"][item] for item in range(3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [43]:
# we can check our labels padded to the maximum length of the batch with -100
batch["labels"]

tensor([[   60,  7418,  5244,  8234,   740,  4993,     8,  6471,     5,  2218,
            29,   193,  2220,   742,     3,  4366, 14237,    14,     6, 16600,
           301,   548,   301,   331,     5,   193, 24275,    17,     8,   668,
          6142,     3, 33640,    36,    81,     6,  5411,  2709,  9376,    22,
         24275,    59,    36,    19,  9376,   153,   402, 29033, 13774,   402,
         29033,   416,    27,     8,  4034,  4888,     3,     0],
        [  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,

In [44]:
# and decoder input ids to see that they are shifted versions of the labels
batch["decoder_input_ids"]

tensor([[59513,    60,  7418,  5244,  8234,   740,  4993,     8,  6471,     5,
          2218,    29,   193,  2220,   742,     3,  4366, 14237,    14,     6,
         16600,   301,   548,   301,   331,     5,   193, 24275,    17,     8,
           668,  6142,     3, 33640,    36,    81,     6,  5411,  2709,  9376,
            22, 24275,    59,    36,    19,  9376,   153,   402, 29033, 13774,
           402, 29033,   416,    27,     8,  4034,  4888,     3],
        [59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,

In [58]:
# here are the labels for the first and second elements of our dataset
print([tokenized_datasets["train"][i]["labels"] for i in range(1, 3)])
display(JSON(tokenizer.batch_decode([batch["input_ids"][i] for i in range(1, 3)], skip_special_tokens=True)))
display(JSON(tokenizer.batch_decode([batch["labels"][i] for i in range(1,3)], skip_special_tokens=True)))

[[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0], [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]]


<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

Next we take a look at the metric

In [59]:
# SacreBLEU is a popular library to compute the BLEU score
! pip install sacrebleu

                                              0.0/118.9 kB ? eta -:--:--
     --------------------                    61.4/118.9 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 118.9/118.9 kB 1.4 MB/s eta 0:00:00
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached lxml-4.9.2-cp311-cp311-win_amd64.whl (3.8 MB)


In [60]:
# we then load it via evaluate.load()

import evaluate

metric = evaluate.load("sacrebleu")

In [62]:
# Let's try an example - this ia a good translation example with high bleu score
predictions = [
    "This plugin lets you translate web pages between several langauages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]

metric.compute(predictions=predictions, references=references)

{'score': 36.67625610580775,
 'counts': [10, 5, 3, 2],
 'totals': [12, 11, 10, 9],
 'precisions': [83.33333333333333, 45.45454545454545, 30.0, 22.22222222222222],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [63]:
# Let's try another example - this is a bad translation example with low bleu score
predictions = [
    "This the the the the"
]
references = [
    [
    "This plugin allows you to automatically translate web pages between several languages."
    ]
]

metric.compute(predictions=predictions, references=references)

{'score': 2.156693969393992,
 'counts': [1, 0, 0, 0],
 'totals': [5, 4, 3, 2],
 'precisions': [20.0, 12.5, 8.333333333333334, 6.25],
 'bp': 0.20189651799465538,
 'sys_len': 5,
 'ref_len': 13}

In [64]:
# another example with a bad translation and low bleu score
predictions = [
    "This plugin"
]

references = [
    [
    "This plugin allows you to automatically translate web pages between several languages."
    ]
]

metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

The score can go from 0 to 100, and higher is better

To get from the model outputs to texts the metric can use, we will use the tokenizer.batch_decode() method. We just have to clean up all the -100s in the labels (the tokenizer will automatically do the same for the padding token)

In [65]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]
        
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

Fine tuning the model

In [66]:
# log in to hf
from huggingface_hub import notebook_login
notebook_login()

In [68]:
# we can now define our training arguments - hyperparameters for the training
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-fr",
    evaluation_strategy="no", # this will take a while and hence not set
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True, # this is needed to generate the translations by the decoder
    fp16=True, # speeds up training in modern GPUs
    push_to_hub=True,
)

In [69]:
# instantiate the trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/RajkNakka/marian-finetuned-kde4-en-fr into local empty directory.


In [70]:
# before we train let's takke a look at the model's current performance on the validation set
trainer.evaluate(max_length=max_length)

  0%|          | 0/329 [00:00<?, ?it/s]

{'eval_loss': 1.6964491605758667,
 'eval_bleu': 39.16748044596104,
 'eval_runtime': 821.3136,
 'eval_samples_per_second': 25.591,
 'eval_steps_per_second': 0.401}